<a href="https://colab.research.google.com/github/ayk-anonymous/Technical-Assessment-1/blob/main/Cadag_241012_Technical_Assessment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install transformers torch nltk flask

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import json

# Load the dataset
with open('/content/drive/MyDrive/Intent-Chatbot-Dataset.json', 'r') as f:
    dataset = json.load(f)

print(dataset)


[{'user_input': "What's the weather like today?", 'intent': 'get_weather', 'entities': {'date': 'today'}}, {'user_input': 'Book a flight to New York tomorrow.', 'intent': 'book_flight', 'entities': {'destination': 'New York', 'date': 'tomorrow'}}, {'user_input': 'Set a reminder for 9 AM.', 'intent': 'set_reminder', 'entities': {'time': '9 AM'}}, {'user_input': 'Order me a pizza.', 'intent': 'order_food', 'entities': {'food_item': 'pizza'}}, {'user_input': 'Find me a nearby coffee shop.', 'intent': 'find_place', 'entities': {'place_type': 'coffee shop', 'location': 'nearby'}}, {'user_input': 'Play some jazz music.', 'intent': 'play_music', 'entities': {'genre': 'jazz'}}, {'user_input': 'Show me the news.', 'intent': 'get_news', 'entities': {}}, {'user_input': "What's the capital of France?", 'intent': 'get_fact', 'entities': {'country': 'France'}}, {'user_input': "What's 25 times 6?", 'intent': 'math_operation', 'entities': {'operation': 'multiplication', 'numbers': [25, 6]}}, {'user_in

In [10]:
from transformers import pipeline

intent_classifier = pipeline("zero-shot-classification")
entity_extractor = pipeline("ner", aggregation_strategy="simple")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

In [11]:
def identify_intent(user_query):
    intent_labels = [entry['intent'] for entry in dataset]
    classification_result = intent_classifier(user_query, candidate_labels=intent_labels)
    return classification_result

def identify_entities(user_query):
    extracted_entities = entity_extractor(user_query)
    return extracted_entities

In [12]:
feedback_records = []

def gather_feedback(user_query, user_rating):
    feedback_records.append({"query": user_query, "rating": user_rating})

def refine_responses():
    for record in feedback_records:
        if record["rating"] < 3:
            print(f"Enhance response for: {record['query']}")

In [14]:
def generate_chatbot_response(user_query):
    intent_analysis = identify_intent(user_query)
    entities_extraction = identify_entities(user_query)

    top_intent = intent_analysis['labels'][0]
    top_score = intent_analysis['scores'][0]

    extracted_entities = [{"word": entity['word'], "score": entity['score']} for entity in entities_extraction]

    response_message = f"Intent: {top_intent}\nEntities: {extracted_entities}"

    return response_message

user_query = "Make a reservation for two at an Italian restaurant in downtown."
response_message = generate_chatbot_response(user_query)
print(response_message)

user_rating = int(input("Rate the response from 1 to 5: "))
gather_feedback(user_query, user_rating)
refine_responses()

Intent: make_reservation
Entities: [{'word': 'Italian', 'score': 0.9980433}]
Rate the response from 1 to 5: 5
